In [1]:
import os
from src.rdb import get_rdb_string
KAGGLE =  os.getenv("KAGGLE_URL_BASE") is not None
COLAB = os.getenv("COLAB_GPU") is not None
TPU = os.getenv("XRT_TPU_CONFIG") is not None
LOCAL = not KAGGLE and not COLAB

if not LOCAL:
    !git clone https://github.com/nclibz/MRKnee/

if COLAB:
    os.chdir('/content/MRKnee/')
    !git checkout v3
    from google.colab import drive
    drive.mount('/content/drive')
    DATADIR = "/content/drive/MyDrive/MRKnee/data"
    if TPU:
        !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

if KAGGLE:
    os.chdir('/kaggle/working/MRKnee/')
    !git checkout v3
    dataset_name = os.listdir('/kaggle/input')[0]
    rdb_string = 
    DATADIR = f"/kaggle/input/{dataset_name}/"
    MODELDIR = DATADIR
    
    if TPU:
        !pip install torchtext==0.9
        !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
        !python pytorch-xla-env-setup.py --version 1.8

if not LOCAL:
    !pip install --quiet "pytorch-lightning>=1.4.9" "torchmetrics>=0.5" "timm" "neptune-client" "optuna" "PyMySql"
    !pip install albumentations --upgrade --quiet
    BACKBONE = "tf_efficientnetv2_s_in21k"

if LOCAL:
    DATADIR = "data"
    MODELDIR = "src/"
    BACKBONE = 'tf_mobilenetv3_small_minimal_100'
    %load_ext autoreload
    %autoreload 2

from src.model import MRKnee
from src.data import OAI, MRNet

from src.augmentations import Augmentations
from src.callbacks import Callbacks
from src.cfg import Cfg
import pytorch_lightning as pl
from torch.utils.data import DataLoader
pl.seed_everything(123)

Global seed set to 123


123

In [2]:
DIAGNOSIS = "meniscus"
PLANE = "sagittal"

# TODO: mrknee public
# TODO: paths til models 
# TODO: indsæt params fra baseline

model = MRKnee.load_from_checkpoint(
    f"{MODELDIR}models/v3/{DIAGNOSIS}_{PLANE}.ckpt",
    backbone="tf_efficientnetv2_s_in21k",
    drop_rate=0.5,
    learning_rate=1e-4,
    adam_wd=0.001,
    max_epochs=20,
    precision=32,
    log_auc=False,
    log_ind_loss=False,
)

augs = Augmentations(
    train_imgsize= (256,256),
    test_imgsize= (256,256),
    shift_limit=0.1,
    scale_limit=0.1,
    rotate_limit=0.1,
    ssr_p=0.1,
    clahe_p=0.1,
    reverse_p=0.0,
    indp_normalz=True,
)

train_ds = OAI(
    stage="train",
    diagnosis=DIAGNOSIS,
    plane=PLANE,
    clean=False,
    transforms=augs,
    imgs_in_ram = True
)

train_dl = DataLoader(
            train_ds,
            batch_size=1,
            shuffle=True,
            num_workers=1,
            pin_memory=True,
        )


val_ds = OAI(
    stage="valid",
    diagnosis=DIAGNOSIS,
    plane=PLANE,
    clean=False,
    transforms=augs,
    imgs_in_ram = True
)


val_dl = DataLoader(
            val_ds,
            batch_size=1,
            shuffle=False,
            num_workers=1,
            pin_memory=True,
        )


configs = Cfg(model = model, train_ds = train_ds, val_ds = val_ds, augs = augs)
cfg = configs.get_cfg()
callbacks = Callbacks(cfg, neptune_name='nclibz/oai')

trainer = pl.Trainer(
    gpus=1,
    precision=cfg["precision"],
    max_epochs=cfg["max_epochs"],
    logger=callbacks.get_neptune_logger(),
    log_every_n_steps=100,
    num_sanity_val_steps=0,
    callbacks=callbacks.get_callbacks(),
    progress_bar_refresh_rate=20,
    deterministic=False,
)

trainer.fit(model, train_dataloaders=train_dl, val_dataloaders= val_dl)

callbacks.neptune_logger.experiment.stop()



/home/nicolai/mambaforge/envs/dl/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


https://ui.neptune.ai/nclibz/mrkneev3/e/MRKNEEV-539
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


/home/nicolai/mambaforge/envs/dl/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type         | Params
------------------------------------------
0 | backbone | EfficientNet | 20.2 M
1 | clf      | Linear       | 1.3 K 
2 | val_auc  | AUROC        | 0     
------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total p

Epoch 0:   0%|          | 0/1480 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 0; 3.82 GiB total capacity; 2.15 GiB already allocated; 31.44 MiB free; 2.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF